In [321]:
%matplotlib inline
import numpy as np
import quantstats as qs
import pandas as pd
import yfinance as _yf
from pandas.tseries.offsets import MonthEnd

# Override function to use adjusted close, not Close
# https://github.com/ranaroussi/quantstats/blob/main/quantstats/utils.py#L233
def download_returns(ticker, period="max", proxy=None):
    params = {
        "tickers": ticker,
        "proxy": proxy,
    }
    if isinstance(period, pd.DatetimeIndex):
        params["start"] = period[0]
    else:
        params["period"] = period

    returns = _yf.download(**params)["Adj Close"].pct_change()
    return returns 

# extend pandas functionality with metrics, etc.
qs.extend_pandas()

# fetch the daily returns for a stock
# ^IRX: Risk Free Rate (daily or annualized?) (based on 3-Month US Treasury Bills Rates): https://gist.github.com/ranaroussi/72d0e92bbe31d1514baccf00175049e4
stocks = [(
    # qs.utils.download_returns(t)
    download_returns(t)
    .reset_index()
    .assign(returns=lambda df: 1+df['Adj Close'])
    .drop(columns=['Adj Close'])
    .set_index(['Date'])
    .resample('ME')
    .prod()
    .assign(returns=lambda df: df['returns']-1)
    .reset_index()
    .assign(id=t)
    ) for t in ['SPY', 'SSO']]
stocks += [
    (
        (_yf.download("^IRX")["Adj Close"]/100)
        .reset_index()
        .assign(returns=lambda df: (1+df['Adj Close']) ** (1/252))
        .drop(columns=['Adj Close'])
        .set_index(['Date'])
        .resample('ME')
        .apply(lambda df: df['returns'].prod() - 1)
        .reset_index()
        .rename(columns={0: 'returns'})
        .assign(id='IRX')
    )
]
# stock.name = 'returns'
# stock2 = qs.utils.download_returns('BIL')
# stock2.name = 'returns'
stocks[1].tail(3)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,returns,id
212,2024-02-29,0.098965,SSO
213,2024-03-31,0.060207,SSO
214,2024-04-30,-0.108488,SSO


In [324]:
# data_filename_socgen = "lc_investing/data/socgen_trend_index_returns.xlsx"
data_filename_socgen = "c:\\Users\\Emile\\Documents\\lifecycle_investing\\lc_investing\\data\\socgen_trend_index_returns.xlsx"
trend_socgen = (
    pd.read_excel(data_filename_socgen)
    .drop(columns=['index level'])
    .rename(columns={'SG Trend Index': 'returns'})
    .assign(Date=lambda df: df['Date'] - MonthEnd(1))
    .assign(id='SG')
    # .set_index('Date')
)
data = (
    pd.concat(stocks+[trend_socgen], axis=0)
    .pivot(index='Date', columns='id', values='returns')
    .assign(IRX=lambda df: df['IRX']+ (0.0050/12))  # add 50 basis points to borrowing rate due to this blog post: https://www.returnstacked.com/return-stacking-and-the-cost-of-leverage/
    .assign(stocks_trend_m100_100_100=lambda df: (df[['IRX', 'SG', 'SPY']]*np.array([-1,1,1])).sum(axis=1))
    # .assign(stocks_trend_0_100_100=lambda df: (df[['IRX', 'SG', 'SPY']]*np.array([0,1,1])).sum(axis=1)) # zero borrowing cost
    .assign(stocks_trend_m100_0_200=lambda df: (df[['IRX', 'SG', 'SPY']]*np.array([-1,0,2])).sum(axis=1))
)
data1 = (
    data
    .drop(columns=['SPY', 'IRX'])
    .dropna()
)
print(data.tail(2))
data1.tail(3)

id               IRX        SG       SPY       SSO  stocks_trend_m100_100_100  \
Date                                                                            
2024-03-31  0.004469  0.040084  0.032702  0.060207                   0.068317   
2024-04-30  0.003454       NaN -0.053358 -0.108488                  -0.056812   

id          stocks_trend_m100_0_200  
Date                                 
2024-03-31                 0.060935  
2024-04-30                -0.110170  


id,SG,SSO,stocks_trend_m100_100_100,stocks_trend_m100_0_200
Date,,,,
2024-01-31,0.018186,0.024896,0.029454,0.027195
2024-02-29,0.064375,0.098965,0.112094,0.099906
2024-03-31,0.040084,0.060207,0.068317,0.060935


In [325]:
qs.reports.html(data1, benchmark="SPY")

[*********************100%%**********************]  1 of 1 completed
c:\Users\Emile\mambaforge\envs\lcenv1\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\Emile\mambaforge\envs\lcenv1\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\Emile\mambaforge\envs\lcenv1\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scal